In [1]:
import os
import dotenv
import json
import random
import pprint

import openai
from langchain.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser, StructuredOutputParser, ResponseSchema
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.callbacks import get_openai_callback

In [2]:
dotenv.load_dotenv()

True

In [3]:
api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
choices_examples = json.load(open("new_choices_examples.json", "r"))
next_situation_examples = json.load(open("next_situation_examples.json", "r"))

# Choices


In [5]:
choices_chat = ChatOpenAI(model="gpt-4-1106-preview", api_key=api_key, max_tokens=200)

In [6]:
class Choices(BaseModel):
    choice_1: str = Field(..., alias="Choice 1")
    choice_2: str = Field(..., alias="Choice 2")
    choice_3: str = Field(..., alias="Choice 3")
    choice_4: str = Field(..., alias="Choice 4")

In [7]:
choices_parser = PydanticOutputParser(pydantic_object=Choices)

In [8]:
choices_example_template = PromptTemplate(
    input_variables= ["previous_situation", "choices"],
    template="previous situation: {previous_situation}\nchoice 1: {choice_1}\nchoice 2: {choice_2}\nchoice 3: {choice_3}\nchoice 4: {choice_4}",   
)

# choices_fewshot =  FewShotPromptTemplate(
#     examples=random.sample(choices_examples, 2),
#     example_prompt=choices_example_template,
#     suffix="previous situation: {input}",
#     input_variables=["input"],
# )

In [9]:
choices_system_prompt_base = """
You are the AI of the 'Story Twist' novel writing app. In this app, your task is to provide users with four choices to influence the plot of their novel. Your role is to analyze the current plot and generate compelling choices that can take the story in different directions.

Rules:

- Analyze the previous situation of the novel.
- Generate four distinct choices that could logically follow from the current plot.
- Each choice should offer a unique direction for the story's progression.
- Ensure the choices are diverse and cater to different story outcomes.
- Provide the choices in a clear and concise manner for the user to select from."
- Ensure the choices are brief and to the point, less than 40 characters each.

{format_instruction}

Now, let's start!

previous situation: {input} 
"""

choices_system_template = PromptTemplate(
    input_variables=["previous_situation", "format_instruction", "input"],
    template=choices_system_prompt_base,
)

In [10]:
text = """
IT is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered as the rightful property of some one or other of their daughters.

“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”

Mr. Bennet replied that he had not.

“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”

Mr. Bennet made no answer.

“Do not you want to know who has taken it?” cried his wife, impatiently.

“You want to tell me, and I have no objection to hearing it.”
"""

In [11]:
with get_openai_callback() as cb:
    message = choices_system_prompt_base.format(
    format_instruction=choices_parser.get_format_instructions(),
    input=text    
    )
    output = choices_chat(messages=[SystemMessage(content=message)], )
    choices = choices_parser.parse(output.content)
    print(choices)
    print(cb.total_cost)

choice_1='Mr. Bennet ignores her.' choice_2='Mr. Bennet shows interest.' choice_3='Mrs. Bennet keeps the secret.' choice_4='A visitor interrupts them.'
0.0075


In [12]:
output.content

'```json\n{\n  "Choice 1": "Mr. Bennet ignores her.",\n  "Choice 2": "Mr. Bennet shows interest.",\n  "Choice 3": "Mrs. Bennet keeps the secret.",\n  "Choice 4": "A visitor interrupts them."\n}\n```'

# Next Story


In [13]:
class NextStory(BaseModel):
    content: str = Field(..., description="Next situation of the story")

In [14]:
next_story_parser = PydanticOutputParser(pydantic_object=NextStory)

In [17]:
next_story_response_schema = [
    ResponseSchema(name="next_story", type="string", description="Next situation of the story"),
]

In [19]:
next_story_output_parser = StructuredOutputParser.from_response_schemas(response_schemas=next_story_response_schema)

In [49]:
next_situation_system_prompt = """
You are the AI designed for the 'Story Continuation' function in a novel writing app. Your task is to craft the 'next story' of a story by taking into account the 'previous story' and the user's 'choice'. You will synthesize this information to create a seamless and engaging continuation of the plot.

Input:

previous_story: The last known state or event in the story, as written by the user.
choice: The specific direction the user has chosen to take the story.

Output:

Generate a 'next story' that logically follows from the 'previous story' and the 'choice'.

Rules:

- The 'next story' should be a direct continuation that flows naturally from the 'choice'.
- Maintain the tone and style of the original story, ensuring consistency in narrative voice.
- Keep the 'next story' concise yet descriptive enough to inspire the user for further writing.
- Avoid introducing new characters or elements that significantly deviate from the established plot.
- The length of the 'next story' must not exceed 300 characters.
- Ensure that the 'next situation' opens possibilities for further plot development.
- Focus on advancing the plot or character development in a meaningful way.

{format_instruction}

Now, let's start!

previous_story: {previous_story}
choice: {choice}
"""

next_story_template = PromptTemplate(
    input_variables=["format_instruction" , "previous_story", "choice"],
    template=next_situation_system_prompt,
)

In [50]:
choices

Choices(choice_1='Mr. Bennet ignores her.', choice_2='Mr. Bennet shows interest.', choice_3='Mrs. Bennet keeps the secret.', choice_4='A visitor interrupts them.')

In [51]:
next_story_chat = ChatOpenAI(model="gpt-4-1106-preview", api_key=api_key, max_tokens=200, temperature=0.9)

In [52]:
with get_openai_callback() as cb:
    next_story_instruction = next_story_template.format(format_instruction=next_story_output_parser.get_format_instructions(), previous_story=text, choice=choices.choice_4)
    output = next_story_chat(messages=[SystemMessage(content=next_story_instruction)], )
    next_story =  next_story_output_parser.parse(output.content)
    print(next_story)
    print(cb.total_cost)

{'next_story': "The clamor of a knock interrupted them. Mr. Bennet frowned, 'It seems we have a visitor,' he said, rising to answer the door."}
0.00627


In [47]:
output.content

'```json\n{\n\t"next_story": "A knock interrupted their repartee. Mrs. Hill bustling in, \'Sir, a gentleman to see you.\'"\n}\n```'